In [1]:
import pandas as pd 
import math
import numpy as np 
import yfinance as yf 
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import os
import time
import tejapi

# 下載股票資料

In [2]:
api_key = 'YOUR KEY'
tejapi.ApiConfig.api_key = api_key
tejapi.ApiConfig.ignoretz = True
gte, lte = '2019-01-01', '2023-01-01'
data = tejapi.get('TWN/APRCD',
                   paginate = True,
                   coid = '2330',
                   mdate = {'gte':gte, 'lte':lte},
                   opts = {
                       'columns':[ 'mdate', 'open_d', 'high_d', 'low_d', 'close_d', 'volume']
                   }
                  )

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 標準化

In [5]:
mdate = data.index

In [6]:
scaler = StandardScaler()
data = scaler.fit_transform(data)

# train test split

In [7]:
train, test = data[:int(0.8 * len(data)), :4], data[int(0.8 * len(data)):, :4]

# 建造DATASET

In [9]:
def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback, :]
        target = dataset[i+1:i+lookback+1][-1][-1]

        # 只使用收盤價作為input
        # feature = dataset[i:i+lookback]
        # target = dataset[i+1:i+lookback+1][-1]

        X.append(feature)
        y.append(target)
    return torch.FloatTensor(X).to(device), torch.FloatTensor(y).view(-1, 1).to(device)

In [10]:
lookback = 5
X_train, y_train = create_dataset(train, lookback = lookback)
X_val, y_val = create_dataset(test, lookback = lookback)
print(X_train.size(), y_train.size())
print(X_val.size(), y_val.size())
loader = DataLoader(TensorDataset(X_train, y_train), shuffle = False, batch_size = 32)

<ipython-input-9-edca643dfc02>:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.FloatTensor(X).to(device), torch.FloatTensor(y).view(-1, 1).to(device)


torch.Size([776, 5, 4]) torch.Size([776, 1])
torch.Size([191, 5, 4]) torch.Size([191, 1])


# single layer lstm

In [11]:
class S_LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size = 4, hidden_size=64, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(64, 1)
    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout(x)
        x = x[:, -1, :]
        x = self.linear(x)
        return x

In [12]:
def trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer):
  train_loss, test_loss = [],[]
  for epoch in range(epochs):
    model.train()
    for batch, (x, y_true) in enumerate(loader):
      y_pred = model(x)
      loss = criterion(y_pred, y_true)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    model.eval()
    with torch.no_grad():
      y_pred = model(X_train)
      train_rmse = np.sqrt(criterion(y_pred, y_train).item())
      train_loss.append(train_rmse)
      y_pred = model(X_val)
      test_rmse = np.sqrt(criterion(y_pred, y_val).item())
      test_loss.append(test_rmse)
      if (epoch+1) % 100 == 0:
        print('epoch %d train rmse %.4f test rmse %.4f' % (epoch+1, train_rmse, test_rmse))
  return train_loss, test_loss

In [13]:
model = S_LSTM().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())
epochs = 1000

In [14]:
start = time.time()
slstm_train_loss, slstm_test_loss = trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer)
end = time.time()
print('single lstm time cost %.4f' %(end-start))

epoch 100 train rmse 0.0846 test rmse 0.0855
epoch 200 train rmse 0.0541 test rmse 0.0790
epoch 300 train rmse 0.0517 test rmse 0.0732
epoch 400 train rmse 0.0566 test rmse 0.0741
epoch 500 train rmse 0.0670 test rmse 0.0700
epoch 600 train rmse 0.0502 test rmse 0.0734
epoch 700 train rmse 0.0670 test rmse 0.0697
epoch 800 train rmse 0.0520 test rmse 0.0705
epoch 900 train rmse 0.0497 test rmse 0.0711
epoch 1000 train rmse 0.0630 test rmse 0.0688
single lstm time cost 69.5436


In [15]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(epochs), y=slstm_train_loss,
                    mode='lines',
                    name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(epochs) , y=slstm_test_loss,
                    mode='lines',
                    name='Validation Loss'))
fig.update_layout(
    title="Loss curve for single lstm",
    xaxis_title="epochs",
    yaxis_title="rmse"
)
fig.show()

In [16]:
train_plot = np.ones_like(data[:, 3]) * np.nan
test_plot = np.ones_like(data[:, 3]) * np.nan
with torch.no_grad():
  y_pred = model(X_train)
  train_plot[lookback:int(0.8 * len(data))] = y_pred.view(-1).cpu()
  y_pred = model(X_val)
  test_plot[int(0.8 * len(data))+lookback:] = y_pred.view(-1).cpu()

fig = go.Figure()
fig.add_trace(go.Scatter(x=mdate, y=train_plot,
                    mode='lines',
                    name='Train'))
fig.add_trace(go.Scatter(x=mdate , y=test_plot,
                    mode='lines',
                    name='Validation'))
fig.add_trace(go.Scatter(x=mdate , y=data[:, 3],
                    mode='lines',
                    name='True'))
fig.update_layout(
    title="Stock prediction for sngle lstm",
    xaxis_title="dates",
    yaxis_title="standardised stock"
)
fig.show()

# stacked lstm

In [17]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size = 4, hidden_size=64, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(0.4)
        self.lstm2 = nn.LSTM(input_size = 64, hidden_size=32, num_layers=1, batch_first=True)
        self.dropout2 = nn.Dropout(0.4)
        self.linear = nn.Linear(32, 1)
    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.dropout2(x)
        x = x[:, -1, :]
        x = self.linear(x)
        return x

In [18]:
def trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer):
  train_loss, test_loss = [],[]
  for epoch in range(epochs):
    model.train()
    for batch, (x, y_true) in enumerate(loader):
      y_pred = model(x)
      loss = criterion(y_pred, y_true)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    model.eval()
    with torch.no_grad():
      y_pred = model(X_train)
      train_rmse = np.sqrt(criterion(y_pred, y_train).item())
      train_loss.append(train_rmse)
      y_pred = model(X_val)
      test_rmse = np.sqrt(criterion(y_pred, y_val).item())
      test_loss.append(test_rmse)
      if (epoch+1) % 100 == 0:
        print('epoch %d train rmse %.4f test rmse %.4f' % (epoch+1, train_rmse, test_rmse))
  return train_loss, test_loss

In [19]:
model = LSTM().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())
epochs = 1000

In [20]:
start = time.time()
lstm_train_loss, lstm_test_loss = trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer)
end = time.time()
print('stack lstm time cost %.4f' %(end-start))

epoch 100 train rmse 0.0785 test rmse 0.1028
epoch 200 train rmse 0.1248 test rmse 0.0944
epoch 300 train rmse 0.0893 test rmse 0.1243
epoch 400 train rmse 0.0675 test rmse 0.0828
epoch 500 train rmse 0.0770 test rmse 0.0784
epoch 600 train rmse 0.0996 test rmse 0.0777
epoch 700 train rmse 0.1242 test rmse 0.0752
epoch 800 train rmse 0.0639 test rmse 0.0744
epoch 900 train rmse 0.1071 test rmse 0.0833
epoch 1000 train rmse 0.0936 test rmse 0.0836
stack lstm time cost 84.4005


In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(epochs), y=lstm_train_loss,
                    mode='lines',
                    name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(epochs) , y=lstm_test_loss,
                    mode='lines',
                    name='Validation Loss'))
fig.update_layout(
    title="Loss curve for stacked lstm",
    xaxis_title="epochs",
    yaxis_title="rmse"
)
fig.show()

In [22]:
train_plot = np.ones_like(data[:, 3]) * np.nan
test_plot = np.ones_like(data[:, 3]) * np.nan
with torch.no_grad():
  y_pred = model(X_train)
  train_plot[lookback:int(0.8 * len(data))] = y_pred.view(-1).cpu()
  y_pred = model(X_val)
  test_plot[int(0.8 * len(data))+lookback:] = y_pred.view(-1).cpu()

fig = go.Figure()
fig.add_trace(go.Scatter(x=mdate, y=train_plot,
                    mode='lines',
                    name='Train'))
fig.add_trace(go.Scatter(x=mdate , y=test_plot,
                    mode='lines',
                    name='Validation'))
fig.add_trace(go.Scatter(x=mdate , y=data[:, 3],
                    mode='lines',
                    name='True'))
fig.update_layout(
    title="Stock prediction for stacked lstm",
    xaxis_title="dates",
    yaxis_title="standardised stock"
)
fig.show()

# Single layer GRU

In [23]:
class S_GRU(nn.Module):
    def __init__(self):
        super().__init__()
        self.gru1 = nn.GRU(input_size = 4, hidden_size=64, num_layers=1, batch_first = True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(64, 1)
    def forward(self, x):
        x, _ = self.gru1(x)
        x = self.dropout(x)
        x = x[:, -1, :]
        x = self.linear(x)
        return x

In [24]:
def trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer):
  train_loss, test_loss = [],[]
  for epoch in range(epochs):
    model.train()
    for batch, (x, y_true) in enumerate(loader):
      y_pred = model(x)
      loss = criterion(y_pred, y_true)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    model.eval()
    with torch.no_grad():
      y_pred = model(X_train)
      train_rmse = np.sqrt(criterion(y_pred, y_train).item())
      train_loss.append(train_rmse)
      y_pred = model(X_val)
      test_rmse = np.sqrt(criterion(y_pred, y_val).item())
      test_loss.append(test_rmse)
      if (epoch+1) % 100 == 0:
        print('epoch %d train rmse %.4f test rmse %.4f' % (epoch+1, train_rmse, test_rmse))
  return train_loss, test_loss

In [25]:
model = S_GRU().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())
epochs = 1000

In [26]:
start = time.time()
sgru_train_loss, sgru_test_loss = trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer)
end = time.time()
print('single gru time cost %.4f' %(end-start))

epoch 100 train rmse 0.0707 test rmse 0.0824
epoch 200 train rmse 0.0533 test rmse 0.0754
epoch 300 train rmse 0.0522 test rmse 0.0710
epoch 400 train rmse 0.0504 test rmse 0.0722
epoch 500 train rmse 0.0573 test rmse 0.0694
epoch 600 train rmse 0.0576 test rmse 0.0690
epoch 700 train rmse 0.0576 test rmse 0.0693
epoch 800 train rmse 0.0548 test rmse 0.0687
epoch 900 train rmse 0.0494 test rmse 0.0713
epoch 1000 train rmse 0.0541 test rmse 0.0689
single gru time cost 58.5323


In [27]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(epochs), y=sgru_train_loss,
                    mode='lines',
                    name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(epochs) , y=sgru_test_loss,
                    mode='lines',
                    name='Validation Loss'))
fig.update_layout(
    title="Loss curve for single gru",
    xaxis_title="epochs",
    yaxis_title="rmse"
)
fig.show()

In [28]:
train_plot = np.ones_like(data[:, 3]) * np.nan
test_plot = np.ones_like(data[:, 3]) * np.nan
with torch.no_grad():
  y_pred = model(X_train)
  train_plot[lookback:int(0.8 * len(data))] = y_pred.view(-1).cpu()
  y_pred = model(X_val)
  test_plot[int(0.8 * len(data))+lookback:] = y_pred.view(-1).cpu()

fig = go.Figure()
fig.add_trace(go.Scatter(x=mdate, y=train_plot,
                    mode='lines',
                    name='Train'))
fig.add_trace(go.Scatter(x=mdate , y=test_plot,
                    mode='lines',
                    name='Validation'))
fig.add_trace(go.Scatter(x=mdate , y=data[:, 3],
                    mode='lines',
                    name='True'))
fig.update_layout(
    title="Stock prediction for single gru",
    xaxis_title="dates",
    yaxis_title="standardised stock"
)
fig.show()

# Stacked layer gru

In [37]:
class GRU(nn.Module):
    def __init__(self):
        super().__init__()
        self.gru1 = nn.GRU(input_size = 4, hidden_size=64, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(0.4)
        self.gru2 = nn.GRU(input_size = 64, hidden_size=32, num_layers=1, batch_first=True)
        self.dropout2 = nn.Dropout(0.4)
        self.linear = nn.Linear(32, 1)
    def forward(self, x):
        x, _ = self.gru1(x)
        x = self.dropout1(x)
        x, _ = self.gru2(x)
        x = self.dropout2(x)
        x = x[:, -1, :]
        x = self.linear(x)
        return x

In [38]:
def trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer):
  train_loss, test_loss = [],[]
  for epoch in range(epochs):
    model.train()
    for batch, (x, y_true) in enumerate(loader):
      y_pred = model(x)
      loss = criterion(y_pred, y_true)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    model.eval()
    with torch.no_grad():
      y_pred = model(X_train)
      train_rmse = np.sqrt(criterion(y_pred, y_train).item())
      train_loss.append(train_rmse)
      y_pred = model(X_val)
      test_rmse = np.sqrt(criterion(y_pred, y_val).item())
      test_loss.append(test_rmse)
      if (epoch+1) % 100 == 0:
        print('epoch %d train rmse %.4f test rmse %.4f' % (epoch+1, train_rmse, test_rmse))
  return train_loss, test_loss

In [39]:
model = GRU().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())
epochs = 1000

In [40]:
start = time.time()
gru_train_loss, gru_test_loss = trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer)
end = time.time()
print('stack gru time cost %.4f' %(end-start))

epoch 100 train rmse 0.0717 test rmse 0.0874
epoch 200 train rmse 0.1043 test rmse 0.0824
epoch 300 train rmse 0.0667 test rmse 0.0872
epoch 400 train rmse 0.0883 test rmse 0.0797
epoch 500 train rmse 0.0628 test rmse 0.0800
epoch 600 train rmse 0.0698 test rmse 0.0805
epoch 700 train rmse 0.0545 test rmse 0.0819
epoch 800 train rmse 0.0748 test rmse 0.0815
epoch 900 train rmse 0.0580 test rmse 0.0790
epoch 1000 train rmse 0.0788 test rmse 0.0882
stack gru time cost 81.2973


In [45]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(epochs), y=gru_train_loss,
                    mode='lines',
                    name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(epochs) , y=gru_test_loss,
                    mode='lines',
                    name='Validation Loss'))
fig.update_layout(
    title="Loss curve for stacked gru",
    xaxis_title="epochs",
    yaxis_title="rmse"
)
fig.show()

In [44]:
train_plot = np.ones_like(data[:, 3]) * np.nan
test_plot = np.ones_like(data[:, 3]) * np.nan
with torch.no_grad():
  y_pred = model(X_train)
  train_plot[lookback:int(0.8 * len(data))] = y_pred.view(-1).cpu()
  y_pred = model(X_val)
  test_plot[int(0.8 * len(data))+lookback:] = y_pred.view(-1).cpu()

fig = go.Figure()
fig.add_trace(go.Scatter(x=mdate, y=train_plot,
                    mode='lines',
                    name='Train'))
fig.add_trace(go.Scatter(x=mdate , y=test_plot,
                    mode='lines',
                    name='Validation'))
fig.add_trace(go.Scatter(x=mdate , y=data[:, 3],
                    mode='lines',
                    name='True'))
fig.update_layout(
    title="Stock prediction for stacked gru",
    xaxis_title="dates",
    yaxis_title="standardised stock"
)
fig.show()

# loss comparison

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(epochs) , y=slstm_test_loss,
                    mode='lines',
                    name='single lstm validation Loss'))


fig.add_trace(go.Scatter(x=np.arange(epochs) , y=sgru_test_loss,
                    mode='lines',
                    name='single gru validation Loss'))


fig.show()

In [35]:
fig = go.Figure()
# fig.add_trace(go.Scatter(x=np.arange(epochs), y=slstm_train_loss,
#                     mode='lines',
#                     name='single lstm train Loss'))
fig.add_trace(go.Scatter(x=np.arange(epochs) , y=slstm_test_loss,
                    mode='lines',
                    name='single lstm validation Loss'))

# fig.add_trace(go.Scatter(x=np.arange(epochs), y=lstm_train_loss,
#                     mode='lines',
#                     name='stacked lstm train Loss'))
# fig.add_trace(go.Scatter(x=np.arange(epochs) , y=lstm_test_loss,
#                     mode='lines',
#                     name='stacked lstm validation Loss'))

# fig.add_trace(go.Scatter(x=np.arange(epochs), y=sgru_train_loss,
#                     mode='lines',
#                     name='single gru train Loss'))
fig.add_trace(go.Scatter(x=np.arange(epochs) , y=sgru_test_loss,
                    mode='lines',
                    name='single gru validation Loss'))

# fig.add_trace(go.Scatter(x=np.arange(epochs), y=gru_train_loss,
#                     mode='lines',
#                     name='stacked gru train Loss'))
# fig.add_trace(go.Scatter(x=np.arange(epochs) , y=gru_test_loss,
#                     mode='lines',
#                     name='stacked gru validation Loss'))

fig.show()